In [ ]:
# 📦 Install necessary packages
!pip install sentence-transformers faiss-cpu pymupdf python-docx fpdf tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
 

In [ ]:
import zipfile
import os

# Create a directory to extract the files
extract_dir = '/content/'
os.makedirs(extract_dir, exist_ok=True)

# Path to the zip file
zip_file_path = 'data/data.zip'

# Check if the zip file exists before attempting to extract
if os.path.exists(zip_file_path):
  try:
    # Open the zip file in read mode
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      # Extract all contents to the specified directory
      zip_ref.extractall(extract_dir)
    print(f"Successfully extracted {zip_file_path} to {extract_dir}")
  except zipfile.BadZipFile:
    print(f"Error: {zip_file_path} is not a valid zip file.")
  except FileNotFoundError:
    print(f"Error: {zip_file_path} not found.")
else:
  print(f"Error: {zip_file_path} not found.")

# Check if the extracted directory is empty
if not os.listdir(extract_dir):
    print(f"Warning: The directory {extract_dir} is empty. Please ensure the zip file was extracted correctly or that the files were uploaded to the correct location.")
else:
    # You can list the contents of the extracted directory to verify
    print("\nContents of extracted directory:")
    !ls {extract_dir}

Successfully extracted data.zip to /content/

Contents of extracted directory:
data.zip  RAG  sample_data


In [ ]:
import os
from pathlib import Path
import fitz  # PyMuPDF
import docx
import re
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from tqdm import tqdm
from fpdf import FPDF

In [ ]:
# 📂 Path to uploaded files - Now pointing to the extracted directory
base_path = Path("/content/RAG")
# Ensure the directory exists (it should if extraction was successful)
if not base_path.exists():
    print(f"Error: The directory {base_path} does not exist. Please ensure the zip file was extracted correctly.")
    # Exit or handle the error appropriately
    exit()


# 📄 Function to read .pdf files
def read_pdf(path):
    doc = fitz.open(path)
    return "\n".join([page.get_text() for page in doc])

# 📄 Function to read .docx files
def read_docx(path):
    doc = docx.Document(path)
    return "\n".join([para.text for para in doc.paragraphs])

100%|██████████| 127/127 [00:06<00:00, 19.19it/s]


Created 1089 chunks.


,file,chunk_id,text
0,Data structures and algorithms/סיכומים/סיכומים...,0,אביטל נבו גיא אלעד 41 2 41 תDescribe Place Tim...
1,Data structures and algorithms/סיכומים/סיכומים...,1,1 lg n Heapify A i 1 בונה ערימת מקס מינ נעזר ב...
2,Data structures and algorithms/סיכומים/סיכומים...,2,n Preorder שורש תת עץ שמאלי תת עץ ימני Inorder...
3,Data structures and algorithms/סיכומים/סיכומים...,3,לא קשירים V E DFS G 9 רקורסיבי V DFS_Visit u מ...
4,Data structures and algorithms/סיכומים/סיכומים...,4,אם היא בין שני קודקודים מקבוצות שונות מאחדים א...


In [ ]:
import re
import docx

# --- Read DOCX ---
def read_docx(path):
    doc = docx.Document(path)
    return "\n".join([para.text for para in doc.paragraphs])

# --- Chunk by paragraphs, max_words_per_chunk to keep semantic chunks ---
def chunk_text_by_paragraphs(text, max_words_per_chunk=300):
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    chunks = []
    current_chunk = []
    current_word_count = 0

    for para in paragraphs:
        para_word_count = len(re.findall(r'\b\w+\b', para))
        if current_word_count + para_word_count > max_words_per_chunk:
            if current_chunk:
                chunks.append("\n\n".join(current_chunk))
            current_chunk = [para]
            current_word_count = para_word_count
        else:
            current_chunk.append(para)
            current_word_count += para_word_count

    if current_chunk:
        chunks.append("\n\n".join(current_chunk))
    return chunks

# --- Usage example ---
docx_file_path = '/content/docx example.docx'  # Replace with your file path

docx_content = read_docx(docx_file_path)
chunks = chunk_text_by_paragraphs(docx_content)

print(f"✅ Created {len(chunks)} chunks.\n")

for i, chunk in enumerate(chunks[:3]):  # show first 3 chunks raw
    print(f"--- Chunk {i} ---")
    print(chunk)
    print("\n" + "-"*40 + "\n")


✅ Created 49 chunks.

--- Chunk 0 ---
מבני נתונים ואלגוריתמים- הקלטות של יובל עמק
סיכום של רוב ההרצאות (חלק לא הספקתי לסיים להקליד)
הרצאה 1 26.10.20
אלגוריתם הוא נכון אם הוא עוצר עם פלט נכון עבור כל קלט.
אלגוריתם לא נכון יכול לעצור עם פלט לא נכון או לא לעצור כלל עבור פלטים מסויימים.
איך אנחנו מודדים את האיכות של האלגוריתם (הנכון)?
זמן ריצה- ההתמקדות העיקרית שלנו 
דרישות זיכרון
אלגוריתמים מתוארים ע"י שימוש בפסדו-קוד.
הפסדו-קוד גמיש ו"סלחן", אך מעורפל.
בקלות מתורגם לכל השפות.

----------------------------------------

--- Chunk 1 ---
אנו נתעלם בקורס מהיררכיות זיכרון, garbage collection ועוד.
הפעולות בהן נשתמש: מבוצעות על מספר סופי של איברים.
פעולות אריתמטיות: 
פעולות לוגיות: 
bitwise operators: 
program control: if,for,goto,while..
במה אסור להשתמש: average, sort, find min…
משום שלא מובן בפעולות אלו האם מדובר במספר סופי של איברים.
ההנחה שאנחנו עושים היא שכל פעולה לוקחת מספר קבוע של פעולות זמן.
בזמן ריצה, אנו מעוניינים בworst-case, המקרה הגרוע ביותר.
הבטחה גדולה יותר למשתמש לגבי האלגוריתם


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 351.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.3 MB/s eta 0:00:00


In [ ]:
!apt install tesseract-ocr-heb tesseract-ocr-eng poppler-utils
!pip install pytesseract pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
The following NEW packages will be installed:
  poppler-utils tesseract-ocr-heb
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 618 kB of archives.
After this operation, 1,673 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-heb all 1:4.00~git30-7274cfa-1.1 [432 kB]
Fetched 618 kB in 2s (267 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Selecting previously unselec

In [ ]:
!pip install multilingual-pdf2text==1.1.0

In [ ]:
!pip install python-bidi arabic_reshaper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 1.0 MB/s eta 0:00:00


In [ ]:
import re
import os
from pdfminer.high_level import extract_text
from bidi.algorithm import get_display
from IPython.display import display, HTML
import pdfplumber
from pdfminer.high_level import extract_text
# Optional: if your PDFs contain Arabic script that needs reshaping
# import arabic_reshaper

def remove_cid_artifacts(text):
    return re.sub(r'\(cid:\d+\)', '', text)

def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

def read_pdf_with_pdfplumber(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def print_bidi_text(text):
    html = f"""
    <div dir="auto" style="white-space: pre-wrap;
                          font-family: sans-serif;
                          font-size: 14px;
                          line-height: 1.5;">
        {text}
    </div>
    """
    display(HTML(html))

def is_hebrew_or_arabic(char):
    # Hebrew range: \u0590-\u05FF
    # Arabic range: \u0600-\u06FF
    return ('\u0590' <= char <= '\u05FF') or ('\u0600' <= char <= '\u06FF')

def process_line_bidi(line):
    # Apply BiDi algorithm only if line contains Hebrew or Arabic chars
    if any(is_hebrew_or_arabic(ch) for ch in line):
        # For Arabic, you may need reshaping:
        # reshaped_text = arabic_reshaper.reshape(line)
        # return get_display(reshaped_text)
        return get_display(line)
    else:
        return line

def process_text_bidi(text):
    lines = text.splitlines()
    processed_lines = [process_line_bidi(line) for line in lines]
    return "\n".join(processed_lines)

# Replace with your PDF file path
pdf_path = "/content/pdf example.pdf"

if os.path.exists(pdf_path):
    #raw_text = read_pdf_with_pdfplumber(pdf_path)
    raw_text = extract_text(pdf_path)
    raw_text = remove_cid_artifacts(raw_text)
    cleaned_text = process_text_bidi(raw_text)
    print(cleaned_text)
    #print_bidi_text(cleaned_text)
else:
    print(f"File not found: {pdf_path}")


# --- Step 1: Chunk the raw text (before BiDi) ---
def chunk_text_by_paragraphs(text, max_words_per_chunk=300):
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    chunks = []
    current_chunk = []
    current_word_count = 0

    for para in paragraphs:
        para_word_count = len(re.findall(r'\b\w+\b', para))
        if current_word_count + para_word_count > max_words_per_chunk:
            if current_chunk:
                chunks.append("\n\n".join(current_chunk))
            current_chunk = [para]
            current_word_count = para_word_count
        else:
            current_chunk.append(para)
            current_word_count += para_word_count

    if current_chunk:
        chunks.append("\n\n".join(current_chunk))
    return chunks

# --- Step 2: Create and print chunks ---
chunks = chunk_text_by_paragraphs(cleaned_text)
print(f"\n✅ Found {len(chunks)} chunks.\n")

for i, chunk in enumerate(chunks[:3]):  # Print first 3 for inspection
    print(f"--- Chunk {i} ---")
    print(chunk)
    print("\n" + "-"*40 + "\n")


מבני נתונים ואלגוריתמים (094224) ־־ חורף תש״ף

שאלה 1 (שאלה מתרגיל בית)
(35 נקודות)
יהי (G = (V, E גרף מכוון עם פונקציית משקל w : E → R≥0 על הקשתות. תהי {b : E → {0, 1 פונקציה
בוליאנית שמתאימה ביט (0 או 1) לכל קשת בגרף. מסלול v0, v1, . . . , vk ב G נקרא מסלול מתחלף אם מתקיים
(b(vi−1, vi) = b(vi, vi+1 לכל i ≤ k − 1 ≥ 1. כלומר, הביטים של הקשתות לאורך המסלול מתחלפים לסרוגין בין 0
ו 1. בהינתן שני צמתים u, v ∈ V, נסמן ב (δb(u, v את משקלו של מסלול מתחלף קל ביותר מ u ל v ב G, כאשר
∞ = (δb(u, v אם לא קיים מסלול כזה.
b : E → {0, 1} פונקציה בוליאנית ,w : E → R≥0 תכנני אלגוריתם שמקבל כקלט גרף מכוון פשוט, פונקציית משקל
.O((n + m) log n) זמן הריצה של האלגוריתם צריך להיות .δb(s, v) את v ∈ V ומחשב לכל צומת ,s ∈ V וצומת מקור
אין חובה לספק פסאודוקוד. יש להסביר את נכונות האלגוריתם וסיבוכיות הזמן.

1


מבני נתונים ואלגוריתמים (094224) ־־ חורף תש״ף

שאלה 2
(35 נקודות)
v ∈ V במילים אחרות, לכל .k חסומה על־ידי קבוע v ∈ V גרף מכוון שבו הדרגה היוצאת של כל צומת G = (V, E) יהי
.k = O(1) ו degout(v) ≤ k מתקיים

ה

In [ ]:
import re
import os
import docx
import logging
import pandas as pd
import pdfplumber
from pathlib import Path
from bidi.algorithm import get_display
from tqdm.notebook import tqdm
import concurrent.futures
import unicodedata
from pdfminer.high_level import extract_text

# --- Suppress pdfminer warnings ---
logging.getLogger("pdfminer.pdfpage").setLevel(logging.ERROR)
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# --- Remove (cid:xx) artifacts ---
def remove_cid_artifacts(text):
    return re.sub(r'\(cid:\d+\)', '', text)

# --- Hebrew/Arabic BiDi helpers ---
def is_hebrew_or_arabic(char):
    return ('\u0590' <= char <= '\u05FF') or ('\u0600' <= char <= '\u06FF')

def sanitize_text_for_bidi(text):
    # Remove control chars except \n, \r, \t and normal printable chars
    # Unicode category 'C' = Other (including control chars)
    return ''.join(ch for ch in text if ch == '\n' or ch == '\r' or ch == '\t' or (32 <= ord(ch) <= 0x10FFFF and not unicodedata.category(ch).startswith('C')))


def process_line_bidi(line):
    line = sanitize_text_for_bidi(line)
    if any(is_hebrew_or_arabic(ch) for ch in line):
        try:
            return get_display(line)
        except AssertionError as e:
            print(f"⚠️ Bidi assertion error for line: {line[:30]}... skipping bidi fix")
            return line
    else:
        return line

def process_text_bidi(text):
    lines = text.splitlines()
    processed_lines = [process_line_bidi(line) for line in lines]
    return "\n".join(processed_lines)

# --- File reading functions with bidi and cleanup ---
def read_pdf(file_path):
    try:
        text = extract_text(file_path)
        cleaned = remove_cid_artifacts(text)
        return cleaned
    except Exception as e:
        print(f"❌ pdfminer failed on {file_path}, error: {e}")
        return ""


def read_docx(file_path):
    try:
        doc = docx.Document(file_path)
        raw_text = "\n".join([para.text for para in doc.paragraphs])
        return raw_text
    except Exception as e:
        print(f"❌ python-docx failed on {file_path}, error: {e}")
        return ""


# --- Chunking text ---
def chunk_text(text, chunk_size=300):
    words = re.findall(r'\b\w+\b', text)
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

def chunk_text_by_paragraphs(text, max_words_per_chunk=300):
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    chunks = []
    current_chunk = []
    current_word_count = 0

    for para in paragraphs:
        para_word_count = len(re.findall(r'\b\w+\b', para))
        if current_word_count + para_word_count > max_words_per_chunk:
            if current_chunk:
                chunks.append("\n\n".join(current_chunk))
            current_chunk = [para]
            current_word_count = para_word_count
        else:
            current_chunk.append(para)
            current_word_count += para_word_count

    if current_chunk:
        chunks.append("\n\n".join(current_chunk))
    return chunks


# --- Path setup ---
base_path = Path("/content/RAG")  # Change to your root directory
all_files = list(base_path.rglob("*"))
all_chunks = []

# --- Process each file in parallel ---
def process_file(file):
    if file.is_file() and file.suffix.lower() in ['.pdf', '.docx']:
        try:
            if file.suffix.lower() == '.pdf':
                text = read_pdf(file)
            else:
                text = read_docx(file)
            return file, text
        except Exception as e:
            print(f"❌ Error reading {file}: {e}")
            return file, ""
    return None, ""

base_path = Path("/content/RAG")  # Update as needed
all_files = list(base_path.rglob("*"))
all_chunks = []

pbar = tqdm(all_files, desc="Starting", total=len(all_files))

for file in pbar:
    if file.is_file() and file.suffix.lower() in ['.pdf', '.docx']:
        pbar.set_description(f"Processing {file.name}")  # Truncate name

        if file.suffix.lower() == '.pdf':
            text = read_pdf(file)
            if not text.strip():
                continue
            text = process_text_bidi(text)  # Apply BiDi only to PDFs

        else:
            text = read_docx(file)
            if not text.strip():
                continue
            # NO BiDi for DOCX, keep text as-is

        chunks = chunk_text_by_paragraphs(text)

        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "file": str(file.relative_to(base_path)),
                "chunk_id": i,
                "text": chunk
            })

pbar.close()

df_chunks = pd.DataFrame(all_chunks)
print(f"\n✅ Created {len(df_chunks)} chunks.")
display(df_chunks.head())

Starting:   0%|          | 0/127 [00:00<?, ?it/s]


✅ Created 895 chunks.


,file,chunk_id,text
0,Data structures and algorithms/סיכומים/סיכומים...,0,41/2/41\n\nName\nInsertion_Sort\n\nMerge\n\nMe...
1,Data structures and algorithms/סיכומים/סיכומים...,1,מתבסס על שתי הפונקציות שלהלן.\nn \nפעם אחת בונ...
2,Data structures and algorithms/סיכומים/סיכומים...,2,חיפוש האם ערך קיים בעץ חיפוש בינארי.\n/ \n** א...
3,Data structures and algorithms/סיכומים/סיכומים...,3,כאשר אח של הצומת הוא אדום.\n\nר\nונעלה במעלה ה...
4,Data structures and algorithms/סיכומים/סיכומים...,4,"MST_Kurskal(g,w)\n)לפי קשתות(\n\nFind_set(u)\n..."


In [ ]:
df_chunks

,file,chunk_id,text
0,Data structures and algorithms/סיכומים/סיכומים...,0,41/2/41\n\nName\nInsertion_Sort\n\nMerge\n\nMe...
1,Data structures and algorithms/סיכומים/סיכומים...,1,מתבסס על שתי הפונקציות שלהלן.\nn \nפעם אחת בונ...
2,Data structures and algorithms/סיכומים/סיכומים...,2,חיפוש האם ערך קיים בעץ חיפוש בינארי.\n/ \n** א...
3,Data structures and algorithms/סיכומים/סיכומים...,3,כאשר אח של הצומת הוא אדום.\n\nר\nונעלה במעלה ה...
4,Data structures and algorithms/סיכומים/סיכומים...,4,"MST_Kurskal(g,w)\n)לפי קשתות(\n\nFind_set(u)\n..."
...,...,...,...
890,Visual information display/הגורם האנושי באיסוף...,5,"גם כאשר המדגם איכותי, יש\nלנתונים\nכדאי לקצר א..."
891,Visual information display/הגורם האנושי באיסוף...,0,שחזור מבחן הגורם האנושי - אנא כתבו את השאלות ש...
892,Visual information display/הגורם האנושי באיסוף...,1,שאלה\nבטכניון רוצים לבחון את השפעת ההוראה מרחו...
893,Visual information display/הגורם האנושי באיסוף...,2,2.??\n3.??\n4.??\nאף.5\nכל.6\n\nמהתשובות\nנכונ...


In [ ]:
import re

def clean_text_for_chunking(text):
    print(text)
    # 1. Strip spaces from start/end of each line
    text = "\n".join(line.strip() for line in text.splitlines())

    # 2. Replace multiple consecutive blank lines with just two newlines (paragraph separator)
    text = re.sub(r'\n\s*\n+', '\n\n', text)

    # 3. Replace multiple dashes with a single dash (optional)
    text = re.sub(r'-{2,}', '-', text)

    # 4. Replace multiple spaces/tabs with a single space
    text = re.sub(r'[^\S\r\n]{2,}', ' ', text)

    # 5. Join lines within paragraphs (paragraphs are separated by blank lines)
    paragraphs = text.split('\n\n')
    paragraphs = [" ".join(p.splitlines()) for p in paragraphs]
    cleaned_text = "\n\n".join(paragraphs)

    return cleaned_text

for i, row in df_chunks.iterrows():
    df_chunks.at[i, 'text'] = clean_text_for_chunking(row['text'])


In [ ]:
df_chunks

,file,chunk_id,text
0,Data structures and algorithms/סיכומים/סיכומים...,0,41/2/41\n\nName Insertion_Sort\n\nMerge\n\nMer...
1,Data structures and algorithms/סיכומים/סיכומים...,1,מתבסס על שתי הפונקציות שלהלן. n פעם אחת בונה ע...
2,Data structures and algorithms/סיכומים/סיכומים...,2,חיפוש האם ערך קיים בעץ חיפוש בינארי. / ** אם ה...
3,Data structures and algorithms/סיכומים/סיכומים...,3,כאשר אח של הצומת הוא אדום.\n\nר ונעלה במעלה הע...
4,Data structures and algorithms/סיכומים/סיכומים...,4,"MST_Kurskal(g,w) )לפי קשתות(\n\nFind_set(u)\n\..."
...,...,...,...
890,Visual information display/הגורם האנושי באיסוף...,5,"גם כאשר המדגם איכותי, יש לנתונים כדאי לקצר את ..."
891,Visual information display/הגורם האנושי באיסוף...,0,שחזור מבחן הגורם האנושי - אנא כתבו את השאלות ש...
892,Visual information display/הגורם האנושי באיסוף...,1,שאלה בטכניון רוצים לבחון את השפעת ההוראה מרחוק...
893,Visual information display/הגורם האנושי באיסוף...,2,2.?? 3.?? 4.?? אף.5 כל.6\n\nמהתשובות נכונות\n\...


In [ ]:
df_chunks.to_csv("df_chunks.csv", index=False)

In [ ]:
from sklearn.preprocessing import normalize

# 🧠 Step 2: Embed with SentenceTransformer
print("🔍 Generating embeddings...")
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df_chunks['text'].tolist(), convert_to_numpy=True)

# ✅ Normalize for cosine similarity
embeddings = normalize(embeddings)

# 🗃️ Step 3: Store in FAISS index (cosine similarity)
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dim)  # Inner Product = cosine when vectors are normalized
index.add(embeddings)

# 💾 Save FAISS index and metadata
faiss.write_index(index, "rag_index.faiss")
df_chunks.to_csv("rag_chunks.csv", index=False)
print("✅ Index & metadata saved.")

🔍 Generating embeddings...
✅ Index & metadata saved.


In [ ]:
# Install the DejaVu font
!apt-get update -qq > /dev/null
!apt-get install -y fonts-dejavu-core -qq > /dev/null
print("DejaVu font installed.")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
DejaVu font installed.


In [ ]:
# 🔎 Step 4: Retrieve top 5 chunks for a user query
def search_query(query: str, top_k=1):
    query_embedding = model.encode([query], convert_to_numpy=True)
    query_embedding = normalize(query_embedding)
    distances, indices = index.search(query_embedding.astype('float32'), top_k)
    results = df_chunks.iloc[indices[0]].copy()
    results["score"] = distances[0]
    return results


def safe_bidi(text):
    """Applies BiDi fix and removes unsupported characters (like emojis)."""
    try:
        # Use BiDi to reverse RTL correctly
        text = get_display(text)
    except Exception:
        pass
    # Remove emojis or problematic symbols (basic filtering)
    text = re.sub(r'[^\x00-\x7F\u0590-\u05FF\u0600-\u06FF\s.,;:?!()"\']', '', text)
    return text

def create_pdf_from_results(results_df, filename="top_chunks.pdf"):
    pdf = FPDF()
    pdf.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font('DejaVu', '', 14)

    for i, row in results_df.iterrows():
        pdf.add_page()
        pdf.set_xy(10, 10)

        header = f"🗂 קובץ: {row['file']}\n🧩 צ'אנק {row['chunk_id']}\n\n"
        body = row['text']

        # Apply BiDi + sanitize for both
        header_fixed = safe_bidi(header)
        body_fixed = safe_bidi(body)

        pdf.multi_cell(0, 10, header_fixed + body_fixed, align='R')

    pdf.output(filename)
    print(f"📄 PDF saved as {filename}")

# 🔁 Example: Run a query and save results to PDF
query = "מה זה BFS?"
results = search_query(query, top_k=5)
for i, row in results.iterrows():
    print(f"\n🔢 Score: {row['score']:.4f}")
    print(f"📁 File: {row['file']}")
    print(f"🔢 Chunk ID: {row['chunk_id']}")
    print(f"🧩 Text: {row['text'][:300]}")


🔢 Score: 0.4733
📁 File: Visual information display/הגורם האנושי באיסוף נתונים - 096275/הרצאות/2020-2021 Winter/Lecture 02 - crowdsourcing.pdf
🔢 Chunk ID: 5
🧩 Text: 32

?

באיזה משימה לא משנה מה יש בהפסקה איזו שיטת הפסקה הכי ? יעילה

באיזו משימה

?

שאלות : •

•

🔢 Score: 0.4356
📁 File: Data structures and algorithms/סיכומים/סיכומים/לפי נושאים/mst.pdf
🔢 Chunk ID: 5
🧩 Text: הם עפ"מים לפי

סתירה.

–

גדול יותר

, משקלו לפי

-ב

מאשר

במשקל

coheng@gmail.com

גיל כהן

0'otherwisewewewwewe01T2T'w1T0w2T'w

🔢 Score: 0.3716
📁 File: Data structures and algorithms/סיכומים/סיכומים/דף עזר למבחן במבני.docx
🔢 Chunk ID: 0
🧩 Text: סיכום אלגוריתמים

סיכום מבני נתונים

🔢 Score: 0.3646
📁 File: Data structures and algorithms/סיכומים/סיכומים/סיכום מבני נתונים 1 - ארוך.pdf
🔢 Chunk ID: 84
🧩 Text: מבני נתונים, 234218, טכניון

אלעד שטייגמן

kui ab] ufcdai yd] lfm ]i favi ambk fndb uv] ivki ofk bf] o anfd ]fa foet vfv g io nnk CS xi] b] ofk bf] ]ii cbudk nfdi lbfoua (fubtka ib)ebufa bvbbvaksoioucbdvcbudkvfvivk